# Validación cruzada


## Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co

## Referencias

1. Alvaro Montenegro, [Curso de Estadística Bayesiana](https://github.com/AprendizajeProfundo/Estadistica-Bayesiana), 2020
2. Guangyuan Gao, "Bayesian Claims Reserving Methods in Non-life Insurance with Stan. An Introduction", Springer, 2018

# Introducción

Cuando los datos no son suficientes para ser divididos en dos grupos: entrenamiento y validación, aún es posible usar en lo modelos de regresión las técnicas basadas en  la distribución predictiva como se decribe en esta lección. Primero, recordemos como generar nuevas observaciones con la distribución predictiva.

# Predición de  datos no observados

Consideremos un modelo bayesiano con parámetros $\boldsymbol{\theta}$ y datos continuos $\mathbf{y}$. Vamos a considerar la posterior $p(\boldsymbol{\theta}|y)$ . Entonces generamos nuevos datos como sigue:

1. Generamos una muestra $\boldsymbol{\theta}^{(1)}, \ldots, \boldsymbol{\theta}^{(n)}$, a partir de la posterior $p(\boldsymbol{\theta}|y)$.

2. Para cada muestra $\boldsymbol{\theta}^{(k)}$ se obtiene una muestra $\tilde{y}_k$ a partir de valores de un vector concido de variables predictoras, digamos $\mathbf{z}$,  y $\boldsymbol{\theta}^{(k)}$. La muestra se obtiene  de la verosimilitud $p(y|\boldsymbol{\theta}^{(k)},\mathbf{z})$.



## Ejemplo. Modelo de Regresión Normal homocedástico $\mathcal{N}(\mu_i,\sigma^2)$

Si este es el modelo verdadero que produjo los datos, lo que ocurrió es que 

### Generación de los datos

Este es el proceso que teóricamente ocurrió para generar los datos.

1. $\boldsymbol{\theta}$ fué generado a partir de una distribución (desconocido) $P(\boldsymbol{\theta})$. En este ejemplo se tiene que  $\boldsymbol{\theta} = (\alpha,\boldsymbol{\beta},\sigma)'$.

2. Cada observación $y_i$ tienen distribución $\mathcal{N}(\mu_i,\sigma^2)$, en donde $\mu_i = \alpha + \mathbf{x}_i'\boldsymbol{\beta}$. Así que el dato observado $y_i$ es obtenido como una muestra aleatoria de $\mathcal{N}(\mu_i,\sigma^2)$. 


### Generación de nuevos datos

Nosotros construimos la posterior $p(\boldsymbol{\theta}|y)$. Esta es nuestra aproximación de la distribución desconocida $P(\boldsymbol{\theta})$. Entonces los datos simulados no observadosse obtienen como sigue.

1. Se obtiene una muestra $\boldsymbol{\theta}^{(1)}, \ldots, \boldsymbol{\theta}^{(n)}$, a partir de la posterior $p(\boldsymbol{\theta}|y)$.  Aquí $\boldsymbol{\theta}^{(k)} =(\alpha^{(k)}, \boldsymbol{\beta}^{(k)}, \sigma^{(k)})'$.

2. A partir de cada muestra posterior $(\alpha^{(k)}, \boldsymbol{\beta}^{(k)}, \sigma^{(k)})'$ se calcula $\mu_z^{(k)}= \alpha^{(k)} +\mathbf{z}'\boldsymbol{\beta}^{(k)} $. El dato no observado $\tilde{y}$ en este caso se obtiene como una muestra de la distribución $\mathcal{N}(\mu_z^{(k)},  \sigma^{(k)})$.

# Generación de datos para validación cruzada en modelos de regresión (dejando uno por fuera)

Esta  técnica es conocida como *loo* del inglés leave one out, o dejar uno por fuera.

En este caso, el procedimiento a seguir es el siguiente. Sea $\mathbf{y}_{-i}$ el conjunto de observaciones excepto $y_i$. Sea $\mathbf{x}_{i}$  el vector de varables predictoras asociadas a $y_i$. Entonces, una réplica de la observación $y_i$ se obtiene como sigue.

1. Se obtiene una muestra $\boldsymbol{\theta}^*_{-i}$ de la posterior $p(\boldsymbol{\theta}|\mathbf{y}_{-i})$.
2. Se obtiene una réplica de  $y_i$ como una muestra  de la verosimilitud $p(y|\boldsymbol{\theta}^*_{-i},\mathbf{x}_{i})$.

## Validación cruzada

Lo que se hace es generar muestras *loo*. Sea  $\boldsymbol{\theta}^*_{-i}$ una muestra de la posterior $p(\boldsymbol{\theta}|\mathbf{y}_{-i})$. Definimos $\mu_{-i}$ como la media aosciada al problema, entonces



### Residuales de Pearson estandarizados loo

$$
\begin{align}
r_{loo}(i) &= \frac{(y_i-\mu_{-i})}{\sigma_{-i}}\\
T(\mathbf{y}|\boldsymbol{\theta}_{-i}) &= \sum_{i=1}^n \frac{(y_i-\mu_{-i})}{\sigma_{-i}}
\end{align}
$$

###   Discrepancia Deviance 

$$
\begin{align}
T(y_i|\boldsymbol{\theta}_{-i}) &= -2 \log f(y_i|\boldsymbol{\theta}_{-i})\\ 
T(\mathbf{y}|\boldsymbol{\theta}_{-i})&=-2 \sum_{i=1}^n \log f(y_i|\boldsymbol{\theta}_{-i})
\end{align}
$$



# Detección de valores extremos (outliers)

El p-valor Bayesiano puede usarse para detectar valores extremos de los datos, de acuerdo con la distribución del problema.

La idea clave es la siguiente. Defina un unmbral, digamos $\text{umbral} = 2.5$a  partir del cual, si se tiene una variable estandarizada, un valor mayor a este umbral puede considerarse sospechoso de ser un valor extremo.

1. En cada paso del algoritmo de muestreo calcule el residual. es decir
$$
r_i^{(k)} = \frac{y_i-\mu_{-i}^{(k)}}{\sigma_{-i}^{(k)}}
$$
2. Calcule la estadística $1_{\{|r_i^{(k)}|>\text{umbral}\}}$
3. Si se obtienen $m$ muestras, el p-valor Bayesiano es dado por

$$
p_{\text{outlier}}(y_i) = \frac{1}{m}\sum_{k=1}^m 1_{\{|r_i^{(k)}|>\text{umbral}\}}
$$
 
Entonces, valores grandes de $p_{\text{outlier}}(y_i)$ señalan a $y_i$ como un valor extremo, con una probabilidad dada por este p-valor Bayesiano.

# Implementación en Stan

En esta sección implementamos los conceptos de la lección para los datos stack_loss de lecciones previas. Usaremos el modelo normal.

In [2]:
import numpy as np
import pystan
%load_ext stanmagic
%matplotlib inline
import numpy as np
import pylab as pl

In [201]:
%%stan -f normal_stack_loss_loo.stan
// Linear Model with normal Errors

data {
  int<lower=0> N;  // number of observations
  int<lower=0> p;  // number of explained variables
  vector[N] y;     // observations
  matrix[N,p] x;   // design matrix
  real umbral;     // threshold for outlier detection
  real y_out;      // y loo
  vector[p] x_out;      // x loo
} 

transformed data {
  matrix[N,p] z;
  vector[p] mean_x;
  vector[p] sd_x;
    
  for (j in 1:p) { 
    mean_x[j] = mean(col(x,j)); 
    sd_x[j] = sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] = (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
}

parameters {
  real beta_0; 
  vector[p] beta; 
  real<lower=0> sigma; 
} 

//transformed parameters {
//    vector[N] mu;
//    mu = beta_0 + z*beta;
//}

model {
  vector[N] mu;
  mu = beta_0 + z*beta; 
  //  priors  
  beta_0 ~ normal(0, 5); 
  beta   ~ normal(0, 5);
  sigma  ~ cauchy(0, 2);
  // likelihood
  y      ~ normal(mu, sigma);
} 

generated quantities {
  real residual; // Pearson residual
  real outlier;  // test for outlier
  real resi_dev; // residual deviance 
  real y_rep;    // replica
  real mu_out;    // mu loo
  real ppo;
  //
  mu_out = dot_product(x_out,beta) + beta_0;
  residual = (y_out - mu_out) / sigma;
  outlier  = step(residual - umbral);
  // cambiar la siguiente línea en cada caso
  resi_dev = (y_out - mu_out) / sigma;
  // cambiar la siguientes líneas en cada caso
  y_rep  = normal_rng(mu_out,sigma);
  ppo = exp(normal_lpdf(y_out|mu_out,sigma)+10);   
}    

Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in _stan_model object.
Type _stan_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
_stan_model.model_file -> Name of stan_file [normal_stack_loss_loo.stan]
_stan_model.model_name -> Name of stan model [normal_stack_loss_loo_model]
_stan_model.model_code -> Model code [// Linear Model with ....]


In [202]:
_stan_model

model_file: normal_stack_loss_loo.stan
model_name: normal_stack_loss_loo_model
model_code:
// Linear Model with normal Errors

data {
  int<lower=0> N;  // number of observations
  int<lower=0> p;  // number of explained variables
  vector[N] y;     // observations
  matrix[N,p] x;   // design matrix
  real umbral;     // threshold for outlier detection
  real y_out;      // y loo
  vector[p] x_out;      // x loo
} 

transformed data {
  matrix[N,p] z;
  vector[p] mean_x;
  vector[p] sd_x;
    
  for (j in 1:p) { 
    mean_x[j] = mean(col(x,j)); 
    sd_x[j] = sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] = (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
}

parameters {
  real beta_0; 
  vector[p] beta; 
  real<lower=0> sigma; 
} 

//transformed parameters {
//    vector[N] mu;
//    mu = beta_0 + z*beta;
//}

model {
  vector[N] mu;
  mu = beta_0 + z*beta; 
  //  priors  
  beta_0 ~ normal(0, 5); 
  beta   ~ normal(0, 5);
  sigma  ~ cauchy(0, 2);
  // likelihood
  y      ~ normal(mu, sigma);
} 

generated quantities {
  real residual; // Pearson residual
  real outlier;  // test for outlier
  real resi_dev; // residual deviance 
  real y_rep;    // replica
  real mu_out;    // mu loo
  real ppo;
  //
  mu_out = dot_product(x_out,beta) + beta_0;
  residual = (y_out - mu_out) / sigma;
  outlier  = step(residual - umbral);
  // cambiar la siguiente línea en cada caso
  resi_dev = (y_out - mu_out) / sigma;
  // cambiar la siguientes líneas en cada caso
  y_rep  = normal_rng(mu_out,sigma);
  ppo = exp(normal_lpdf(y_out|mu_out,sigma)+10);   
}

In [203]:
normal_stack_loss_reg_loo = pystan.StanModel(file=_stan_model.model_file)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_eb6353e4f178ab2d7770c2ed41433290 NOW.


# Genera muestras para validación cruzada

In [193]:
# datos
import numpy as np
p = 3
N = 21

y = np.array([43, 37, 37, 28, 18, 18, 19, 20, 15, 14, 14, 13, 11, 12, 8, 
7, 8, 8, 9, 15, 15])
x = np.resize((80, 80, 75, 62, 62, 62, 62, 62, 59, 58, 58, 58, 58, 
58, 50, 50, 50, 50, 50, 56, 70, 27, 27, 25, 24, 22, 23, 24, 24, 
23, 18, 18, 17, 18, 19, 18, 18, 19, 19, 20, 20, 20, 89, 88, 90, 
87, 87, 87, 93, 93, 87, 80, 89, 88, 82, 93, 89, 86, 72, 79, 80, 
82, 91), (21, 3))



# Ajustes con validación cruzada

In [204]:
import pandas as pd

# number of values in the output
num_rows = 12
# threshold to p_value based on residuals
umbral = 2.0
np.random.seed(500) # rep

# list to save the data frames
fit_list =[]

for i in range(21):
    # prepare data
    y_data = np.hstack([y[:i],y[(i+1):]])
    y_out = y[i]
    x_data = np.delete(x,i,axis=0)
    x_out = x[i,:]
    x_out
    stacks_data = {'p': p, 'N': N-1, 'umbral':umbral, 
               'y': y_data,
               'x': x_data,
               'y_out':y_out,
               'x_out':x_out,}
    # fit loo
    sample = normal_stack_loss_reg_loo.sampling(data=stacks_data, iter =4000)
    # extract data from summary
    summary = sample.summary()
    colnames = np.array(summary['summary_colnames'])
    colnames = np.hstack(['id',colnames])
    rownames = summary['summary_rownames']
    data = summary['summary']
    # add a column to identify the index of loo
    id = np.array([i]*num_rows).reshape(num_rows ,1)
    data = np.hstack([id,data])
    # create a data frame
    loo_data = pd.DataFrame(np.round(data,2))
    loo_data.index =rownames
    loo_data.columns = colnames
    # put in the list
    fit_list.append(loo_data)

In [184]:
len(fit_list)

21

In [216]:
i=3
fit_list[i]

,id,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
beta_0,3.0,13.63,0.03,2.47,8.40,12.06,13.77,15.33,18.13,5954.66,1.0
beta[1],3.0,0.86,0.05,3.99,-6.94,-1.85,0.82,3.58,8.74,5923.58,1.0
beta[2],3.0,0.03,0.05,4.07,-7.96,-2.69,0.03,2.73,8.12,5611.58,1.0
beta[3],3.0,-0.89,0.05,3.94,-8.39,-3.61,-0.85,1.77,6.76,6223.81,1.0
sigma,3.0,11.19,0.03,2.08,8.00,9.72,10.88,12.37,16.19,5245.61,1.0
residual,3.0,1.28,0.14,13.00,-24.10,-7.44,1.19,10.05,26.76,8965.95,1.0
outlier,3.0,0.47,0.01,0.50,0.00,0.00,0.00,1.00,1.00,8838.60,1.0
resi_dev,3.0,1.28,0.14,13.00,-24.10,-7.44,1.19,10.05,26.76,8965.95,1.0
y_rep,3.0,14.20,1.56,148.48,-279.87,-81.79,15.69,110.66,304.29,9053.18,1.0
mu_out,3.0,14.04,1.55,147.70,-280.81,-81.62,15.77,110.12,302.81,9100.71,1.0


1. Repita el ejercicio para las distribuciones Laplace y $t_4$
2. ¿Cuáles datos se reportarían como outlier? Distuca su respuesta.
3. ¿Cuál model es aparentemente mejor?

# Tarea